In [1]:
import pandas as pd
from dataset_types import all_up_to_2Mb_dataset_list

# Non augmented

In [2]:
short = pd.read_csv("aug_results/my_short_dataset_results.csv")
short_labeled = pd.read_csv("aug_results/my_short_dataset_results_labeled.csv")
short_labeled.set_index("dataset", drop=True, inplace=True)

In [3]:
results_extra = pd.read_csv("aug_results/results_extra.csv", index_col=0)
results_extra_remaining = pd.read_csv("aug_results/results_extra_remaining.csv", index_col=0)
results_extra_remaining_cuda0 = pd.read_csv("aug_results/results_extra_remaining_cuda0.csv", index_col=0)
results_extra_remaining_cuda1 = pd.read_csv("aug_results/results_extra_remaining_cuda1.csv", index_col=0)
results_extra_remaining_cuda2 = pd.read_csv("aug_results/results_extra_remaining_cuda2.csv", index_col=0)

In [4]:
all_results_extra = pd.concat([results_extra, results_extra_remaining_cuda0, results_extra_remaining_cuda1, results_extra_remaining_cuda2])

In [5]:
all_results = pd.concat([short_labeled, all_results_extra])

In [6]:
original = pd.read_csv("Full_Result/Result_for_128_UCR_datasets.csv")

In [7]:
original

,dataset_name,OS-CNN,OS-CNN-RES(2),OS-CNN-RES(3),OS-CNN-RES(5),OS-CNN-RES(6)
0,ACSF1,90.200000,90.000,94.000,92.000,92.000
1,Adiac,83.452686,82.097,80.307,80.051,79.795
2,AllGestureWiimoteX,80.171429,81.857,78.571,79.000,79.000
3,AllGestureWiimoteY,81.357143,82.286,81.429,81.429,81.143
4,AllGestureWiimoteZ,77.657143,79.000,76.429,74.571,77.286
...,...,...,...,...,...,...
123,Wine,74.444444,79.630,50.000,79.630,22.222
124,WordSynonyms,74.231975,74.765,73.668,71.473,71.944
125,Worms,76.493506,77.922,68.831,61.039,74.026
126,WormsTwoClass,65.714286,74.026,70.130,64.935,76.623


In [8]:
import numpy as np

# orig_vs_all_percentage_columns = [
#     "jitter", "scaling", "window_slice", "window_warp", "time_warp", "rotation",
#     "magnitude_warp", "spawner", "wdba"]
# orig_vs_all_percentage = pd.DataFrame(
#     np.zeros((len(all_up_to_2Mb_dataset_list), len(orig_vs_all_percentage_columns)+1)),
#     index=all_up_to_2Mb_dataset_list, columns=["orig"]+orig_vs_all_percentage_columns
# )
orig_vs_all_percentage = pd.DataFrame()

In [9]:
import numpy as np
for dataset_name in all_up_to_2Mb_dataset_list:
    my_best_idx = all_results[all_results.index.str.startswith(dataset_name)]['mean_acc'].idxmax()
    my_best_val = all_results[all_results.index.str.startswith(dataset_name)]['mean_acc'].max()
    print(f"{dataset_name}. Original: {original.loc[original['dataset_name']==dataset_name, 'OS-CNN'].values[0]} {my_best_idx}: {np.round(my_best_val, 3)}")

Wine. Original: 74.4444443 Wine_rotation: 0.848
BeetleFly. Original: 81.5 BeetleFly_window_slice: 0.92
ArrowHead. Original: 83.7714285 ArrowHead_window_warp: 0.849
OliveOil. Original: 78.6666666 OliveOil_scaling: 0.753
Beef. Original: 80.6666666 Beef_scaling: 0.8
Lightning7. Original: 79.3150687 Lightning7_magnitude_warp: 0.816
Lightning2. Original: 80.6557378 Lightning2_window_warp: 0.852
DistalPhalanxOutlineAgeGroup. Original: 73.8129496 DistalPhalanxOutlineAgeGroup_jitter: 0.771
DistalPhalanxTW. Original: 66.4028775 DistalPhalanxTW_window_slice: 0.694
Herring. Original: 60.78125 Herring_scaling: 0.662
ProximalPhalanxTW. Original: 77.3170733 ProximalPhalanxTW_magnitude_warp: 0.797
MiddlePhalanxOutlineCorrect. Original: 81.4089348 MiddlePhalanxOutlineCorrect_scaling: 0.808
ProximalPhalanxOutlineCorrect. Original: 90.790378 ProximalPhalanxOutlineCorrect_scaling: 0.912
ProximalPhalanxOutlineAgeGroup. Original: 84.3902441 ProximalPhalanxOutlineAgeGroup_wdba: 0.862
DistalPhalanxOutlineCor

In [10]:
import numpy as np

def compute_percent_improvement(orig_value, improv_value):
    return ((improv_value - orig_value) / orig_value) * 100

orig_vs_all_percentage = pd.DataFrame()
for dataset_name in all_up_to_2Mb_dataset_list:
    orig_acc = original.loc[original['dataset_name']==dataset_name, 'OS-CNN'].values[0]/100
    idxs = all_results[all_results.index.str.startswith(dataset_name)]['mean_acc'].index
    idxs = [idx[len(dataset_name)+1:] for idx in idxs]
    accs = all_results[all_results.index.str.startswith(dataset_name)]['mean_acc'].values
    orig_vs_all_percentage.loc[dataset_name, "orig"] = orig_acc
    orig_vs_all_percentage.loc[dataset_name, idxs] = accs
    # print(f"{dataset_name}. Original: {original.loc[original['dataset_name']==dataset_name, 'OS-CNN'].values[0]} {my_best_idx}: {np.round(my_best_val, 3)}")

In [11]:
for column in orig_vs_all_percentage.columns[1:]:
    orig_vs_all_percentage[f"perc_{column}"] = np.round(((orig_vs_all_percentage[column] - orig_vs_all_percentage["orig"]) / orig_vs_all_percentage["orig"]) * 100, 3)

In [12]:
orig_vs_all_percentage.mean()

orig                    0.822632
jitter                  0.793870
scaling                 0.818861
window_slice            0.729584
window_warp             0.781544
time_warp               0.661406
rotation                0.775107
magnitude_warp          0.799964
spawner                 0.703466
wdba                    0.738364
perc_jitter            -3.792577
perc_scaling           -0.580231
perc_window_slice     -12.115192
perc_window_warp       -5.279596
perc_time_warp        -20.216923
perc_rotation          -5.664077
perc_magnitude_warp    -3.034577
perc_spawner          -14.895058
perc_wdba             -11.707385
dtype: float64

# Augmented

In [13]:
aug_cuda3 = pd.read_csv("aug_results/results_aug_cuda3.csv", index_col=0)
aug_remaining_cuda0 = pd.read_csv("aug_results/results_aug_remaining_cuda0.csv", index_col=0)
aug_remaining_cuda0.index = [str(index) + "_aug" for index in aug_remaining_cuda0.index]
aug_remaining_cuda1 = pd.read_csv("aug_results/results_aug_remaining_cuda1.csv", index_col=0)
aug_remaining_cuda1.index = [str(index) + "_aug" for index in aug_remaining_cuda1.index]
aug_remaining_cuda2 = pd.read_csv("aug_results/results_aug_remaining_cuda2.csv", index_col=0)
aug_remaining_cuda2.index = [str(index) + "_aug" for index in aug_remaining_cuda2.index]
aug_remaining_cuda3 = pd.read_csv("aug_results/results_aug_remaining_cuda3.csv", index_col=0)

In [14]:
aug_df = pd.concat([aug_cuda3, aug_remaining_cuda0, aug_remaining_cuda1, aug_remaining_cuda2, aug_remaining_cuda3])

In [15]:
original = pd.read_csv("Full_Result/Result_for_128_UCR_datasets.csv")

In [16]:
aug_df[aug_df.index.str.startswith("Wine")].index

Index(['Wine_jitter_aug', 'Wine_scaling_aug', 'Wine_window_slice_aug',
       'Wine_window_warp_aug', 'Wine_time_warp_aug', 'Wine_rotation_aug',
       'Wine_magnitude_warp_aug', 'Wine_spawner_aug', 'Wine_wdba_aug'],
      dtype='object')

In [17]:
import numpy as np

def compute_percent_improvement(orig_value, improv_value):
    return ((improv_value - orig_value) / orig_value) * 100

orig_vs_aug_percentage = pd.DataFrame()
for dataset_name in all_up_to_2Mb_dataset_list:
    orig_acc = original.loc[original['dataset_name']==dataset_name, 'OS-CNN'].values[0]/100
    idxs = aug_df[aug_df.index.str.startswith(dataset_name)]['mean_acc'].index
    idxs = [idx[len(dataset_name)+1:] for idx in idxs]
    accs = aug_df[aug_df.index.str.startswith(dataset_name)]['mean_acc'].values
    orig_vs_aug_percentage.loc[dataset_name, "orig"] = orig_acc
    orig_vs_aug_percentage.loc[dataset_name, idxs] = accs
    # print(f"{dataset_name}. Original: {original.loc[original['dataset_name']==dataset_name, 'OS-CNN'].values[0]} {my_best_idx}: {np.round(my_best_val, 3)}")
for column in orig_vs_aug_percentage.columns[1:]:
    orig_vs_aug_percentage[f"perc_{column}"] = np.round(((orig_vs_aug_percentage[column] - orig_vs_aug_percentage["orig"]) / orig_vs_aug_percentage["orig"]) * 100, 3)
orig_vs_aug_percentage.mean()

orig                        0.822632
jitter_aug                  0.807831
scaling_aug                 0.820861
window_slice_aug            0.785410
window_warp_aug             0.805695
time_warp_aug               0.731794
rotation_aug                0.782958
magnitude_warp_aug          0.807057
spawner_aug                 0.758801
wdba_aug                    0.770894
perc_jitter_aug            -2.166000
perc_scaling_aug           -0.652865
perc_window_slice_aug      -5.268404
perc_window_warp_aug       -2.366269
perc_time_warp_aug        -10.832115
perc_rotation_aug          -5.021288
perc_magnitude_warp_aug    -2.278058
perc_spawner_aug           -8.221712
perc_wdba_aug              -6.888519
dtype: float64

In [18]:
import numpy as np

dataset_names = all_up_to_2Mb_dataset_list

all_columns = ["jitter", "scaling", "window_slice", "window_warp", "time_warp", "rotation",
                 "magnitude_warp", "spawner", "wdba"]
count_orig_vs_all_columns = ["orig"]
count_orig_vs_all_columns.extend(all_columns)
count_orig_vs_all = pd.DataFrame(np.zeros((len(dataset_names), len(all_columns)+1)), index=dataset_names, columns=count_orig_vs_all_columns)
count_wins_orig_orig_vs_all = 0
count_wins_all_orig_vs_all = 0

aug_columns = ["jitter_aug", "scaling_aug", "window_slice_aug", "window_warp_aug",
               "time_warp_aug", "rotation_aug", "magnitude_warp_aug", "spawner_aug", "wdba_aug"]
count_orig_vs_aug_columns = ["orig"]
count_orig_vs_aug_columns.extend(aug_columns)
count_orig_vs_aug = pd.DataFrame(np.zeros((len(dataset_names), len(aug_columns)+1)), index=dataset_names, columns=count_orig_vs_aug_columns)
count_wins_orig_orig_vs_aug = 0
count_wins_aug_orig_vs_aug = 0

count_aug_vs_all_columns = all_columns + aug_columns
count_aug_vs_all = pd.DataFrame(np.zeros((len(dataset_names), len(count_aug_vs_all_columns))), index=dataset_names, columns=count_aug_vs_all_columns)
count_wins_aug_aug_vs_all = 0
count_wins_all_aug_vs_all = 0

count_orig_vs_all_vs_aug_columns = ["orig"] + all_columns + aug_columns
count_orig_vs_all_vs_aug = pd.DataFrame(np.zeros((len(dataset_names), len(count_orig_vs_all_vs_aug_columns))), index=dataset_names, columns=count_orig_vs_all_vs_aug_columns)
count_wins_orig = 0
count_wins_all = 0
count_wins_aug = 0

for dataset_name in all_up_to_2Mb_dataset_list:
    all_results_best_idx = all_results[all_results.index.str.startswith(dataset_name)]['mean_acc'].idxmax()
    all_results_best_acc = all_results[all_results.index.str.startswith(dataset_name)]['mean_acc'].max()
    aug_best_idx = aug_df[aug_df.index.str.startswith(dataset_name)]['mean_acc'].idxmax()
    aug_best_acc = aug_df[aug_df.index.str.startswith(dataset_name)]['mean_acc'].max()
    orig_acc = original.loc[original['dataset_name']==dataset_name, 'OS-CNN'].values[0]/100

    # Compute percentual accuracy
    
    print(f"{dataset_name}. Original: {orig_acc}" + 
          f" {aug_best_idx}: {np.round(aug_best_acc, 3)}" +
            f" {all_results_best_idx}: {np.round(all_results_best_acc, 3)}")
    if orig_acc > all_results_best_acc:
        count_orig_vs_all.loc[dataset_name, "orig"] = 1
        count_wins_orig_orig_vs_all += 1
        if orig_acc > aug_best_acc:
            count_orig_vs_all_vs_aug.loc[dataset_name, "orig"] = 1
            count_wins_orig += 1
    if orig_acc > aug_best_acc:
        count_orig_vs_aug.loc[dataset_name, "orig"] = 1
        count_wins_orig_orig_vs_aug += 1

    if all_results_best_acc > aug_best_acc:
        # all_results_best_idx[len(dataset_name)+1:] is to remove the name of the dataset and just leave the name of the aug
        count_aug_vs_all.loc[dataset_name, all_results_best_idx[len(dataset_name)+1:]] = 1
        count_wins_all_aug_vs_all += 1
        if all_results_best_acc > orig_acc:
            count_orig_vs_all_vs_aug.loc[dataset_name, all_results_best_idx[len(dataset_name)+1:]] = 1
            count_wins_all += 1
    if all_results_best_acc > orig_acc:
        count_orig_vs_all.loc[dataset_name, all_results_best_idx[len(dataset_name)+1:]] = 1
        count_wins_all_orig_vs_all += 1
    
    if aug_best_acc > orig_acc:
        count_orig_vs_aug.loc[dataset_name, aug_best_idx[len(dataset_name)+1:]] = 1
        count_wins_aug_orig_vs_aug += 1
        if aug_best_acc > all_results_best_acc:
            count_orig_vs_all_vs_aug.loc[dataset_name, aug_best_idx[len(dataset_name)+1:]] = 1
            count_wins_aug += 1
    if aug_best_acc > all_results_best_acc:
        count_aug_vs_all.loc[dataset_name, aug_best_idx[len(dataset_name)+1:]] = 1
        count_wins_aug_aug_vs_all += 1

Wine. Original: 0.744444443 Wine_scaling_aug: 0.837 Wine_rotation: 0.848
BeetleFly. Original: 0.815 BeetleFly_jitter_aug: 0.85 BeetleFly_window_slice: 0.92
ArrowHead. Original: 0.837714285 ArrowHead_window_warp_aug: 0.854 ArrowHead_window_warp: 0.849
OliveOil. Original: 0.786666666 OliveOil_wdba_aug: 0.86 OliveOil_scaling: 0.753
Beef. Original: 0.806666666 Beef_time_warp_aug: 0.827 Beef_scaling: 0.8
Lightning7. Original: 0.793150687 Lightning7_time_warp_aug: 0.819 Lightning7_magnitude_warp: 0.816
Lightning2. Original: 0.8065573779999999 Lightning2_window_slice_aug: 0.849 Lightning2_window_warp: 0.852
DistalPhalanxOutlineAgeGroup. Original: 0.738129496 DistalPhalanxOutlineAgeGroup_magnitude_warp_aug: 0.75 DistalPhalanxOutlineAgeGroup_jitter: 0.771
DistalPhalanxTW. Original: 0.664028775 DistalPhalanxTW_spawner_aug: 0.691 DistalPhalanxTW_window_slice: 0.694
Herring. Original: 0.6078125 Herring_scaling_aug: 0.641 Herring_scaling: 0.662
ProximalPhalanxTW. Original: 0.773170733 ProximalPhala

In [19]:
count_aug_vs_all_columns = ["jit", "scl", "winS", "winW", "timW", "rot", "magW", "spaw", "wdba", "jitA", "sclA", "winSA", "winWA", "timWA", "rotA", "magWA", "spawA", "wdbaA"]
count_aug_vs_all.columns = count_aug_vs_all_columns
count_aug_vs_all.loc["Sum", :] = count_aug_vs_all.sum().values
count_aug_vs_all = count_aug_vs_all.astype(int)
count_aug_vs_all.to_latex()

'\\begin{tabular}{lrrrrrrrrrrrrrrrrrr}\n\\toprule\n & jit & scl & winS & winW & timW & rot & magW & spaw & wdba & jitA & sclA & winSA & winWA & timWA & rotA & magWA & spawA & wdbaA \\\\\n\\midrule\nWine & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nBeetleFly & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nArrowHead & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\\\nOliveOil & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\\\\nBeef & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\\\nLightning7 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\\\nLightning2 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nDistalPhalanxOutlineAgeGroup & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nDistalPhalanxTW & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 

In [20]:
# Table 5
count_orig_vs_all_vs_aug_columns = ["orig", "jit", "scl", "winS", "winW", "timW", "rot", "magW", "spaw", "wdba", "jitA", "sclA", "winSA", "winWA", "timWA", "rotA", "magWA", "spawA", "wdbaA"]
count_orig_vs_all_vs_aug.columns = count_orig_vs_all_vs_aug_columns
count_orig_vs_all_vs_aug.loc["Sum", :] = count_orig_vs_all_vs_aug.sum().values
count_orig_vs_all_vs_aug = count_orig_vs_all_vs_aug.astype(int)
count_orig_vs_all_vs_aug.to_latex()

'\\begin{tabular}{lrrrrrrrrrrrrrrrrrrr}\n\\toprule\n & orig & jit & scl & winS & winW & timW & rot & magW & spaw & wdba & jitA & sclA & winSA & winWA & timWA & rotA & magWA & spawA & wdbaA \\\\\n\\midrule\nWine & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nBeetleFly & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nArrowHead & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\\\nOliveOil & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\\\\nBeef & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\\\nLightning7 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\\\nLightning2 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nDistalPhalanxOutlineAgeGroup & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\nDistalPhalanxTW & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 

In [21]:
# Table 2

wins = count_orig_vs_all_vs_aug.loc["Sum", :]
orig_all_acc = orig_vs_all_percentage.mean().iloc[:10]
aug_acc = orig_vs_aug_percentage.mean().iloc[1:10]
orig_all_aug_acc = pd.concat([orig_all_acc,aug_acc])
orig_all_perc = orig_vs_all_percentage.mean().iloc[10:]
aug_perc = orig_vs_aug_percentage.mean().iloc[10:]
orig_all_aug_perc = pd.concat([pd.Series(0), orig_all_perc, aug_perc])

print(wins)
print(orig_all_acc)
print(aug_acc)
print(orig_all_perc)
print(aug_perc)

wins_acc_perc_df = pd.DataFrame()
wins_acc_perc_df["Wins"] = wins
wins_acc_perc_df["Mean accuracy"] = orig_all_aug_acc.values
wins_acc_perc_df["Accuracy percentual variation"] = orig_all_aug_perc.values

orig     1
jit      4
scl      5
winS     3
winW     2
timW     1
rot      3
magW     3
spaw     1
wdba     0
jitA     8
sclA     2
winSA    1
winWA    3
timWA    5
rotA     2
magWA    1
spawA    1
wdbaA    3
Name: Sum, dtype: int64
orig              0.822632
jitter            0.793870
scaling           0.818861
window_slice      0.729584
window_warp       0.781544
time_warp         0.661406
rotation          0.775107
magnitude_warp    0.799964
spawner           0.703466
wdba              0.738364
dtype: float64
jitter_aug            0.807831
scaling_aug           0.820861
window_slice_aug      0.785410
window_warp_aug       0.805695
time_warp_aug         0.731794
rotation_aug          0.782958
magnitude_warp_aug    0.807057
spawner_aug           0.758801
wdba_aug              0.770894
dtype: float64
perc_jitter            -3.792577
perc_scaling           -0.580231
perc_window_slice     -12.115192
perc_window_warp       -5.279596
perc_time_warp        -20.216923
perc_rotation          

In [32]:
wins_acc_perc_df.to_latex()

'\\begin{tabular}{lrrr}\n\\toprule\n & Wins & Mean accuracy & Accuracy percentual variation \\\\\n\\midrule\norig & 1 & 0.822632 & 0.000000 \\\\\njit & 4 & 0.793870 & -3.792577 \\\\\nscl & 5 & 0.818861 & -0.580231 \\\\\nwinS & 3 & 0.729584 & -12.115192 \\\\\nwinW & 2 & 0.781544 & -5.279596 \\\\\ntimW & 1 & 0.661406 & -20.216923 \\\\\nrot & 3 & 0.775107 & -5.664077 \\\\\nmagW & 3 & 0.799964 & -3.034577 \\\\\nspaw & 1 & 0.703466 & -14.895058 \\\\\nwdba & 0 & 0.738364 & -11.707385 \\\\\njitA & 8 & 0.807831 & -2.166000 \\\\\nsclA & 2 & 0.820861 & -0.652865 \\\\\nwinSA & 1 & 0.785410 & -5.268404 \\\\\nwinWA & 3 & 0.805695 & -2.366269 \\\\\ntimWA & 5 & 0.731794 & -10.832115 \\\\\nrotA & 2 & 0.782958 & -5.021288 \\\\\nmagWA & 1 & 0.807057 & -2.278058 \\\\\nspawA & 1 & 0.758801 & -8.221712 \\\\\nwdbaA & 3 & 0.770894 & -6.888519 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [22]:
orig_vs_all_vs_aug_acc = orig_vs_all_percentage.copy()
orig_vs_all_vs_aug_acc.iloc[:, 10:] = orig_vs_aug_percentage.iloc[:, 1:10]
orig_vs_all_vs_aug_acc["best"] = orig_vs_all_vs_aug_acc.max(axis=1)
print(orig_vs_all_vs_aug_acc.mean()["best"])
compute_percent_improvement(orig_vs_all_vs_aug_acc.mean()["orig"], orig_vs_all_vs_aug_acc.mean()["best"])

0.8550440576405399


3.9400289423529364

In [28]:
orig_all_aug_acc

orig                  0.822632
jitter                0.793870
scaling               0.818861
window_slice          0.729584
window_warp           0.781544
time_warp             0.661406
rotation              0.775107
magnitude_warp        0.799964
spawner               0.703466
wdba                  0.738364
jitter_aug            0.807831
scaling_aug           0.820861
window_slice_aug      0.785410
window_warp_aug       0.805695
time_warp_aug         0.731794
rotation_aug          0.782958
magnitude_warp_aug    0.807057
spawner_aug           0.758801
wdba_aug              0.770894
dtype: float64

In [26]:
print(f"Number wins orig in orig_vs_all: {count_wins_orig_orig_vs_all}")
print(f"Number wins all in orig_vs_all: {count_wins_all_orig_vs_all}")
print("------------------")
print(f"Number wins orig in orig_vs_aug: {count_wins_orig_orig_vs_aug}")
print(f"Number wins aug in orig_vs_aug: {count_wins_aug_orig_vs_aug}")
print("------------------")
print(f"Number wins all in aug_vs_all: {count_wins_all_aug_vs_all}")
print(f"Number wins aug in aug_vs_all: {count_wins_aug_aug_vs_all}")
print("------------------")
print(f"Number wins orig in orig_vs_all_vs_aug: {count_wins_orig}")
print(f"Number wins all in orig_vs_all_vs_aug: {count_wins_all}")
print(f"Number wins aug in orig_vs_all_vs_aug: {count_wins_aug}")
print("------------------")

Number wins orig in orig_vs_all: 9
Number wins all in orig_vs_all: 43
------------------
Number wins orig in orig_vs_aug: 6
Number wins aug in orig_vs_aug: 46
------------------
Number wins all in aug_vs_all: 22
Number wins aug in aug_vs_all: 27
------------------
Number wins orig in orig_vs_all_vs_aug: 1
Number wins all in orig_vs_all_vs_aug: 22
Number wins aug in orig_vs_all_vs_aug: 26
------------------


In [27]:
augmentations = ["jitter", "scaling", "window_slice", "window_warp", "time_warp", "rotation",
                 "magnitude_warp", "spawner", "wdba"]
orig_columns = ["orig"]
orig_columns.extend(augmentations)